In [ ]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import mapping, MultiPoint
import os
from standardize_data import standardize_car_data
import sys
from geopy.distance import geodesic
from datetime import datetime
from shapely.geometry import LineString


In [2]:
# Define the path to the SICAR folder
sicar_folder = "../data/SICAR/2025"

# Initialize an empty list to store GeoDataFrames
sicar_dataframes = []

# Iterate through each state folder in the SICAR directory
for state_folder in os.listdir(sicar_folder):
    state_path = os.path.join(sicar_folder, state_folder)
    if os.path.isdir(state_path):  # Ensure it's a directory
        # Look for shapefiles in the state folder
        for file in os.listdir(state_path):
            if file.endswith(".shp"):  # Check for shapefiles
                file_path = os.path.join(state_path, file)
                # Load the shapefile i nto a GeoDataFrame
                gdf = gpd.read_file(file_path)
                sicar_dataframes.append(gdf)

# Concatenate all GeoDataFrames into one
car_gdf_later_year = gpd.GeoDataFrame(pd.concat(sicar_dataframes, ignore_index=True))

car_gdf_later_year = standardize_car_data(car_gdf_later_year)
car_gdf_later_year = car_gdf_later_year[['cod_imovel','ind_status', 'ind_tipo','cod_estado', 'geometry']]
car_gdf_later_year = car_gdf_later_year[(car_gdf_later_year['ind_tipo'] == 'IRU') & (car_gdf_later_year['ind_status'].isin(['AT', 'PE']))]
car_gdf_later_year = car_gdf_later_year.drop_duplicates()

# remove invalid geometries
car_gdf_later_year = car_gdf_later_year[car_gdf_later_year.geometry.is_valid]

# For testing purposes, limit the number of rows to 500,000
car_gdf_later_year = car_gdf_later_year[:500000]

car_gdf_later_year

,cod_imovel,ind_status,ind_tipo,cod_estado,geometry
2,AC-1200500-FB62BF39455A442591A7D60DBCFA800C,AT,IRU,AC,"POLYGON ((-68.77999 -9.84767, -68.77397 -9.860..."
3,AC-1200708-19C3C6A0A7B6488096185809637AC4AF,AT,IRU,AC,"POLYGON ((-68.19413 -10.70693, -68.19212 -10.7..."
4,AC-1200179-CF396FF9F78E4A0E8CCA9CF80716144A,AT,IRU,AC,"POLYGON ((-67.81002 -10.55802, -67.81534 -10.5..."
5,AC-1200138-218DCFB19D064EF38E435A4281FCEBB4,AT,IRU,AC,"POLYGON ((-68.09258 -9.74467, -68.09028 -9.740..."
6,AC-1200708-E947278AD364400893C00B2C77946904,AT,IRU,AC,"POLYGON ((-68.29565 -10.67157, -68.29556 -10.6..."
...,...,...,...,...,...
573807,PA-1504703-42415B56DBAD4AB6BF39CC1FC5D6BB02,PE,IRU,PA,"POLYGON ((-49.51339 -3.06732, -49.51253 -3.065..."
573808,PA-1500503-2B04952CA267445AA0FB69C26C5B6E9E,PE,IRU,PA,"POLYGON ((-52.74445 -1.56171, -52.74444 -1.561..."
573809,PA-1500503-E19A432528E94328922929E8E4902BB1,PE,IRU,PA,"POLYGON ((-52.73981 -1.53474, -52.73587 -1.539..."
573810,PA-1500503-62C9DD2416154A8E8E84C5EC3FA4DFB1,PE,IRU,PA,"POLYGON ((-52.7552 -1.5524, -52.75273 -1.5515,..."


In [3]:
car_gdf_earlier_year = gpd.read_file("../data/SICAR/2024/")
car_gdf_earlier_year = standardize_car_data(car_gdf_earlier_year)
car_gdf_earlier_year = car_gdf_earlier_year[['cod_imovel','ind_status', 'ind_tipo','cod_estado', 'geometry']]
car_gdf_earlier_year = car_gdf_earlier_year[(car_gdf_earlier_year['ind_tipo'] == 'IRU') & (car_gdf_earlier_year['ind_status'].isin(['AT', 'PE']))]
car_gdf_earlier_year = car_gdf_earlier_year.drop_duplicates()

# remove invalid geometries
car_gdf_earlier_year = car_gdf_earlier_year[car_gdf_earlier_year.geometry.is_valid]

# For testing purposes, limit the number of rows to 500,000
car_gdf_earlier_year = car_gdf_earlier_year[:500000]

car_gdf_earlier_year

/home/jpjon/ccca/myenv/lib/python3.11/site-packages/pyogrio/raw.py:198: RuntimeWarning: ../data/SICAR/2024/merged_car_2024_02_01_UTF8.shp contains polygon(s) with rings with invalid winding order. Autocorrecting them, but that shapefile should be corrected using ogr2ogr for example.
  return ogr_read(


,cod_imovel,ind_status,ind_tipo,cod_estado,geometry
0,AC-1200435-FA1FAFE4584F4A4286756A9DD56A280A,AT,IRU,AC,"POLYGON ((-70.45394 -9.35226, -70.45281 -9.352..."
1,AC-1200435-7966BE27C2484A55A1F2B635C710D9F1,AT,IRU,AC,"POLYGON ((-70.45223 -9.36378, -70.44982 -9.369..."
4,AC-1200435-AF08DCC29D964FBBAB5870C4C46F4288,AT,IRU,AC,"POLYGON ((-70.54442 -9.56007, -70.54881 -9.561..."
5,AC-1200435-AFACFDAA37FC4A789399BE90DE87317E,AT,IRU,AC,"POLYGON ((-69.89842 -9.07162, -69.90032 -9.067..."
6,AC-1200435-C78A60C479494D329B5502FFA8280C00,AT,IRU,AC,"POLYGON ((-70.18799 -9.05937, -70.1693 -9.0602..."
...,...,...,...,...,...
524832,MT-5105150-BC2CC9ECD48C46989A290FD5F7B5C7FE,AT,IRU,MT,"POLYGON ((-58.64119 -11.50074, -58.64047 -11.5..."
524833,MT-5105150-BC425A470D8C4DF990E8937CE8F23F8B,AT,IRU,MT,"POLYGON ((-58.74094 -11.45612, -58.74345 -11.4..."
524836,MT-5105150-BCDAC2DACABB47E6B4A2DFF5FA232A1E,AT,IRU,MT,"POLYGON ((-58.84528 -11.62244, -58.8417 -11.62..."
524837,MT-5105150-BD2DC04E90F148A29FCEF50CEAB6D014,AT,IRU,MT,"POLYGON ((-58.76036 -11.30947, -58.76094 -11.3..."


In [4]:
# Define the path to the PRODES folder
prodes_folder = "../data/PRODES"

# Load the shapefile or GeoJSON file into a GeoDataFrame
# Replace 'filename.shp' with the actual file name in the PRODES folder
prodes_gdf = gpd.read_file(os.path.join(prodes_folder, 'prodes_amazonia_nb.gpkg'))

# Filter only necessary columns in PRODES
prodes_gdf = prodes_gdf[['uuid', 'geometry']]

prodes_gdf = prodes_gdf.to_crs(car_gdf_earlier_year.crs)

/home/jpjon/ccca/myenv/lib/python3.11/site-packages/pyogrio/geopandas.py:265: UserWarning: More than one layer found in 'prodes_amazonia_nb.gpkg': 'accumulated_deforestation_2007_biome' (default), 'hydrography_biome', 'no_forest_biome', 'yearly_deforestation_biome', 'yearly_deforestation_smaller_than_625ha_biome', 'yearly_deforestation_nf_biome', 'residual_biome'. Specify layer parameter to avoid this warning.
  result = read_func(


In [5]:
# Perform the spatial join to find earlier year CAR data that intersects with PRODES data
car_earlier_year_prodes_intersection_gdf = gpd.sjoin(
    car_gdf_earlier_year, prodes_gdf, how="inner", predicate="intersects"
).drop(columns=["index_right"])

In [6]:
car_earlier_year_prodes_intersection_gdf = car_earlier_year_prodes_intersection_gdf.drop_duplicates(subset=['cod_imovel'], keep="first")
car_earlier_year_prodes_intersection_gdf

,cod_imovel,ind_status,ind_tipo,cod_estado,geometry,uuid
0,AC-1200435-FA1FAFE4584F4A4286756A9DD56A280A,AT,IRU,AC,"POLYGON ((-70.45394 -9.35226, -70.45281 -9.352...",09fb5241-0547-44f1-abbe-924719e79a2b
1,AC-1200435-7966BE27C2484A55A1F2B635C710D9F1,AT,IRU,AC,"POLYGON ((-70.45223 -9.36378, -70.44982 -9.369...",bd346c70-a4a9-4c7e-8d2d-ab1a6e5e96ff
4,AC-1200435-AF08DCC29D964FBBAB5870C4C46F4288,AT,IRU,AC,"POLYGON ((-70.54442 -9.56007, -70.54881 -9.561...",74b4512c-e235-4020-8396-2ec623a89e5d
7,AC-1200435-CC642B16CA6C48E58B35AE6B55F4B725,AT,IRU,AC,"POLYGON ((-70.47697 -9.40247, -70.47353 -9.401...",9c59d324-8a61-4c1b-a4fd-a2d598cff4a7
9,AC-1200435-30219B25A3A04D049326EB9E52B89C58,AT,IRU,AC,"POLYGON ((-70.33965 -9.10346, -70.25089 -9.156...",0b4b56b9-3cef-4200-9379-19f11a703213
...,...,...,...,...,...,...
524832,MT-5105150-BC2CC9ECD48C46989A290FD5F7B5C7FE,AT,IRU,MT,"POLYGON ((-58.64119 -11.50074, -58.64047 -11.5...",d0f7e6d5-708e-4454-9278-39e4e7f79be3
524833,MT-5105150-BC425A470D8C4DF990E8937CE8F23F8B,AT,IRU,MT,"POLYGON ((-58.74094 -11.45612, -58.74345 -11.4...",d0f7e6d5-708e-4454-9278-39e4e7f79be3
524836,MT-5105150-BCDAC2DACABB47E6B4A2DFF5FA232A1E,AT,IRU,MT,"POLYGON ((-58.84528 -11.62244, -58.8417 -11.62...",d0f7e6d5-708e-4454-9278-39e4e7f79be3
524837,MT-5105150-BD2DC04E90F148A29FCEF50CEAB6D014,AT,IRU,MT,"POLYGON ((-58.76036 -11.30947, -58.76094 -11.3...",d0f7e6d5-708e-4454-9278-39e4e7f79be3


In [7]:
earlier_year = 2024
later_year=2025

In [8]:
car_later_year_car_early_year_prodes_intersect = car_earlier_year_prodes_intersection_gdf.merge(
    car_gdf_later_year, 
    on="cod_imovel", 
    how="inner", 
    suffixes=(f"_{earlier_year}", f"_{later_year}")
)


In [9]:
# Sitaution where tehre are duplicates from early year to later year, for either because merge has two differnt status' for later year or because lateryear has two duplicates of same cod_imovel
car_later_year_car_early_year_prodes_intersect = car_later_year_car_early_year_prodes_intersect.drop_duplicates(subset=['cod_imovel'], keep="first")

In [10]:
car_later_year_car_early_year_prodes_intersect = car_later_year_car_early_year_prodes_intersect.copy()

car_later_year_car_early_year_prodes_intersect['geometry_changed'] = \
    car_later_year_car_early_year_prodes_intersect.apply(
        lambda row: not row[f'geometry_{earlier_year}'].equals(row[f'geometry_{later_year}']),
        axis=1
    )

In [11]:
# Merge PRODES geometry
car_later_year_car_early_year_prodes_intersect_with_prodes = car_later_year_car_early_year_prodes_intersect.merge(
    prodes_gdf[['uuid', 'geometry']],
    on='uuid',
    how='inner'
).rename(columns={'geometry': 'geometry_prodes'})

In [12]:
# Filter for cadastras that have changed yoy and where they have been changed to no longer intersect with prodes geometries

filtered_indexes = []

for row in car_later_year_car_early_year_prodes_intersect_with_prodes.itertuples():
    if row.geometry_changed and not row.__getattribute__(f'geometry_{later_year}').intersects(row.geometry_prodes):
        filtered_indexes.append(row.Index)

car_later_year_car_early_year_prodes_intersect_with_prodes = car_later_year_car_early_year_prodes_intersect_with_prodes.loc[filtered_indexes]      

car_later_year_car_early_year_prodes_intersect_with_prodes

,cod_imovel,ind_status_2024,ind_tipo_2024,cod_estado_2024,geometry_2024,uuid,ind_status_2025,ind_tipo_2025,cod_estado_2025,geometry_2025,geometry_changed,geometry_prodes
476,AC-1200450-21A692EC02BC424986CF52111EFB4124,AT,IRU,AC,"POLYGON ((-67.50472 -10.07763, -67.52363 -10.0...",cc9cfafa-205b-46e8-8f83-93907ce26b7b,AT,IRU,AC,"POLYGON ((-67.24762 -9.88654, -67.25494 -9.870...",True,"MULTIPOLYGON (((-67.52251 -10.0673, -67.52251 ..."
493,AC-1200450-44542251AAD24ED8B915E4F51ABE33C7,AT,IRU,AC,"POLYGON ((-67.4956 -10.0591, -67.513 -10.0509,...",cc9cfafa-205b-46e8-8f83-93907ce26b7b,AT,IRU,AC,"POLYGON ((-67.26434 -9.87359, -67.25219 -9.888...",True,"MULTIPOLYGON (((-67.52251 -10.0673, -67.52251 ..."
494,AC-1200450-E2A3CC7B400A490C887C0E87DCD431A5,AT,IRU,AC,"POLYGON ((-67.50124 -10.07061, -67.50255 -10.0...",cc9cfafa-205b-46e8-8f83-93907ce26b7b,AT,IRU,AC,"POLYGON ((-67.24404 -9.88522, -67.25116 -9.868...",True,"MULTIPOLYGON (((-67.52251 -10.0673, -67.52251 ..."
707,AC-1200450-CB7E5D7FAAED438B8509D12349A5B0F8,AT,IRU,AC,"POLYGON ((-67.51264 -10.0509, -67.49558 -10.05...",cc9cfafa-205b-46e8-8f83-93907ce26b7b,AT,IRU,AC,"POLYGON ((-67.49119 -10.04993, -67.50657 -10.0...",True,"MULTIPOLYGON (((-67.52251 -10.0673, -67.52251 ..."
708,AC-1200450-B9DAA0D009024986A47E72186D175F6C,AT,IRU,AC,"POLYGON ((-67.51486 -10.05343, -67.49721 -10.0...",cc9cfafa-205b-46e8-8f83-93907ce26b7b,AT,IRU,AC,"POLYGON ((-67.49408 -10.05591, -67.50732 -10.0...",True,"MULTIPOLYGON (((-67.52251 -10.0673, -67.52251 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...
106025,MT-5108303-FDB309AB6E5B497685735484FE4497DD,AT,IRU,MT,"POLYGON ((-54.31139 -11.74656, -54.31511 -11.7...",72e75c55-ec1f-4294-a2ae-543ea242f3ff,AT,IRU,MT,"POLYGON ((-54.32312 -11.71058, -54.32667 -11.7...",True,"MULTIPOLYGON (((-54.32498 -11.74666, -54.32336..."
108601,MT-5108808-24C02CC818A74C1AACADF9BD854E6A22,AT,IRU,MT,"POLYGON ((-55.2337 -10.23539, -55.22818 -10.21...",90e69a15-af8b-4356-a7ef-c5e4a03d168d,AT,IRU,MT,"POLYGON ((-55.2138 -10.24673, -55.21488 -10.24...",True,"MULTIPOLYGON (((-55.20461 -10.21928, -55.20381..."
109218,MT-5108808-3A8531A1BAF24E9191A87D6289B78F34,AT,IRU,MT,"POLYGON ((-55.24926 -10.37358, -55.24603 -10.3...",67ec35e9-6651-4090-8ef9-845ee34be45e,AT,IRU,MT,"POLYGON ((-55.07238 -10.61122, -55.07223 -10.6...",True,"MULTIPOLYGON (((-55.49061 -9.97342, -55.49002 ..."
109418,MT-5108808-E38B8060D0B644E69723682A84EF0316,PE,IRU,MT,"POLYGON ((-55.2331 -10.21075, -55.23228 -10.20...",67ec35e9-6651-4090-8ef9-845ee34be45e,AT,IRU,MT,"POLYGON ((-55.27184 -10.19369, -55.2682 -10.19...",True,"MULTIPOLYGON (((-55.49061 -9.97342, -55.49002 ..."


In [ ]:
# Function to calculate geodesic distance between centroids
def calculate_geodesic_distance(row):
    coord_earlier_year = (row[f'centroid_earlier_year'].y, row['centroid_earlier_year'].x)  # (lat, lon)
    coord_later_year = (row['centroid_later_year'].y, row['centroid_later_year'].x)
    return geodesic(coord_earlier_year, coord_later_year).meters

car_later_year_car_early_year_prodes_intersect_with_prodes[f'centroid_{earlier_year}'] = car_later_year_car_early_year_prodes_intersect_with_prodes[f'geometry_{earlier_year}'].centroid
car_later_year_car_early_year_prodes_intersect_with_prodes[f'centroid_{later_year}'] = car_later_year_car_early_year_prodes_intersect_with_prodes[f'geometry_{later_year}'].centroid


/tmp/ipykernel_35553/1020236105.py:7: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  car_later_year_car_early_year_prodes_intersect_with_prodes['centroid_earlier_year'] = car_later_year_car_early_year_prodes_intersect_with_prodes[f'geometry_{earlier_year}'].centroid
/tmp/ipykernel_35553/1020236105.py:8: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  car_later_year_car_early_year_prodes_intersect_with_prodes['centroid_later_year'] = car_later_year_car_early_year_prodes_intersect_with_prodes[f'geometry_{later_year}'].centroid


In [14]:
car_later_year_car_early_year_prodes_intersect_with_prodes['geodesic_distance'] = car_later_year_car_early_year_prodes_intersect_with_prodes.apply(calculate_geodesic_distance, axis=1)
car_later_year_car_early_year_prodes_intersect_with_prodes

,cod_imovel,ind_status_2024,ind_tipo_2024,cod_estado_2024,geometry_2024,uuid,ind_status_2025,ind_tipo_2025,cod_estado_2025,geometry_2025,geometry_changed,geometry_prodes,centroid_earlier_year,centroid_later_year,geodesic_distance
476,AC-1200450-21A692EC02BC424986CF52111EFB4124,AT,IRU,AC,"POLYGON ((-67.50472 -10.07763, -67.52363 -10.0...",cc9cfafa-205b-46e8-8f83-93907ce26b7b,AT,IRU,AC,"POLYGON ((-67.24762 -9.88654, -67.25494 -9.870...",True,"MULTIPOLYGON (((-67.52251 -10.0673, -67.52251 ...",POINT (-67.5127 -10.07178),POINT (-67.2495 -9.87764),35970.899140
493,AC-1200450-44542251AAD24ED8B915E4F51ABE33C7,AT,IRU,AC,"POLYGON ((-67.4956 -10.0591, -67.513 -10.0509,...",cc9cfafa-205b-46e8-8f83-93907ce26b7b,AT,IRU,AC,"POLYGON ((-67.26434 -9.87359, -67.25219 -9.888...",True,"MULTIPOLYGON (((-67.52251 -10.0673, -67.52251 ...",POINT (-67.5025 -10.0536),POINT (-67.26149 -9.88036),32642.720224
494,AC-1200450-E2A3CC7B400A490C887C0E87DCD431A5,AT,IRU,AC,"POLYGON ((-67.50124 -10.07061, -67.50255 -10.0...",cc9cfafa-205b-46e8-8f83-93907ce26b7b,AT,IRU,AC,"POLYGON ((-67.24404 -9.88522, -67.25116 -9.868...",True,"MULTIPOLYGON (((-67.52251 -10.0673, -67.52251 ...",POINT (-67.51059 -10.06778),POINT (-67.24611 -9.87625),35912.549892
707,AC-1200450-CB7E5D7FAAED438B8509D12349A5B0F8,AT,IRU,AC,"POLYGON ((-67.51264 -10.0509, -67.49558 -10.05...",cc9cfafa-205b-46e8-8f83-93907ce26b7b,AT,IRU,AC,"POLYGON ((-67.49119 -10.04993, -67.50657 -10.0...",True,"MULTIPOLYGON (((-67.52251 -10.0673, -67.52251 ...",POINT (-67.50499 -10.05632),POINT (-67.49981 -10.04404),1472.650583
708,AC-1200450-B9DAA0D009024986A47E72186D175F6C,AT,IRU,AC,"POLYGON ((-67.51486 -10.05343, -67.49721 -10.0...",cc9cfafa-205b-46e8-8f83-93907ce26b7b,AT,IRU,AC,"POLYGON ((-67.49408 -10.05591, -67.50732 -10.0...",True,"MULTIPOLYGON (((-67.52251 -10.0673, -67.52251 ...",POINT (-67.50658 -10.05913),POINT (-67.50058 -10.05064),1146.215382
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106025,MT-5108303-FDB309AB6E5B497685735484FE4497DD,AT,IRU,MT,"POLYGON ((-54.31139 -11.74656, -54.31511 -11.7...",72e75c55-ec1f-4294-a2ae-543ea242f3ff,AT,IRU,MT,"POLYGON ((-54.32312 -11.71058, -54.32667 -11.7...",True,"MULTIPOLYGON (((-54.32498 -11.74666, -54.32336...",POINT (-54.32817 -11.71282),POINT (-54.33436 -11.69356),2234.968900
108601,MT-5108808-24C02CC818A74C1AACADF9BD854E6A22,AT,IRU,MT,"POLYGON ((-55.2337 -10.23539, -55.22818 -10.21...",90e69a15-af8b-4356-a7ef-c5e4a03d168d,AT,IRU,MT,"POLYGON ((-55.2138 -10.24673, -55.21488 -10.24...",True,"MULTIPOLYGON (((-55.20461 -10.21928, -55.20381...",POINT (-55.20702 -10.23247),POINT (-55.2103 -10.23761),672.858810
109218,MT-5108808-3A8531A1BAF24E9191A87D6289B78F34,AT,IRU,MT,"POLYGON ((-55.24926 -10.37358, -55.24603 -10.3...",67ec35e9-6651-4090-8ef9-845ee34be45e,AT,IRU,MT,"POLYGON ((-55.07238 -10.61122, -55.07223 -10.6...",True,"MULTIPOLYGON (((-55.49061 -9.97342, -55.49002 ...",POINT (-55.24913 -10.37698),POINT (-55.07274 -10.61576),32717.645495
109418,MT-5108808-E38B8060D0B644E69723682A84EF0316,PE,IRU,MT,"POLYGON ((-55.2331 -10.21075, -55.23228 -10.20...",67ec35e9-6651-4090-8ef9-845ee34be45e,AT,IRU,MT,"POLYGON ((-55.27184 -10.19369, -55.2682 -10.19...",True,"MULTIPOLYGON (((-55.49061 -9.97342, -55.49002 ...",POINT (-55.2322 -10.21205),POINT (-55.27003 -10.19415),4593.225860


In [31]:
from shapely.geometry import LineString

# can drop iru later to cut down data size

car_later_year_car_early_year_prodes_intersect_with_prodes['distance_line'] = car_later_year_car_early_year_prodes_intersect_with_prodes.apply(
    lambda row: LineString([row[f'geometry_{earlier_year}'].centroid, row[f'geometry_{later_year}'].centroid]),
    axis=1
)

car_later_year_car_early_year_prodes_intersect_with_prodes.head(1)

,cod_imovel,ind_status_2024,ind_tipo_2024,cod_estado_2024,geometry_2024,uuid,ind_status_2025,ind_tipo_2025,cod_estado_2025,geometry_2025,geometry_changed,geometry_prodes,centroid_earlier_year,centroid_later_year,geodesic_distance,distance_line
476,AC-1200450-21A692EC02BC424986CF52111EFB4124,AT,IRU,AC,"POLYGON ((-67.50472 -10.07763, -67.52363 -10.0...",cc9cfafa-205b-46e8-8f83-93907ce26b7b,AT,IRU,AC,"POLYGON ((-67.24762 -9.88654, -67.25494 -9.870...",True,"MULTIPOLYGON (((-67.52251 -10.0673, -67.52251 ...",POINT (-67.5127 -10.07178),POINT (-67.2495 -9.87764),35970.89914,"LINESTRING (-67.5127 -10.07178, -67.2495 -9.87..."


In [ ]:
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Use the timestamp in the output directory
output_dir = f'./outputs/{timestamp}'
os.makedirs(output_dir, exist_ok=True)

fields_to_keep = [
    'cod_imovel', f'ind_status_{later_year}', f'cod_estado_{later_year}',
    'geodesic_distance'
]

geometry_columns = {
    f"geometry_{earlier_year}": f"geometry_{earlier_year}.geojson",
    f"geometry_{later_year}": f"geometry_{later_year}.geojson",
    f"geometry_prodes": "geometry_prodes.geojson",
    f"distance_line": "distance_lines.geojson"
}

for geom_col, filename in geometry_columns.items():
    gdf_out = car_later_year_car_early_year_prodes_intersect_with_prodes[fields_to_keep + [geom_col]].copy()
    gdf_out = gdf_out.set_geometry(geom_col)
    gdf_out.to_file(os.path.join(output_dir, filename), driver="GeoJSON")


/home/jpjon/ccca/myenv/lib/python3.11/site-packages/pyogrio/geopandas.py:662: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(
